In [ ]:
import random

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from gMLV import *

In [ ]:
def set_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
# some plotting functions

cols = ["red", "green", "blue", "royalblue","orange", "black", "salmon", "forestgreen", "steelblue", "slateblue","gold", "palegreen"]

def plot_gMLV(yobs, sobs, timepoints):
    #fig, axs = plt.subplots(1, 2, layout='constrained')
    fig, axs = plt.subplots(1, 2)
    for species_idx in range(yobs.shape[1]):
        axs[0].plot(timepoints, yobs[:, species_idx], color=cols[species_idx])
    axs[0].set_xlabel('time')
    axs[0].set_ylabel('[species]')
    if sobs.shape[1] > 0:
        for metabolite_idx in range(sobs.shape[1]):
            axs[1].plot(timepoints, sobs[:, metabolite_idx], color=cols[metabolite_idx])
        axs[1].set_xlabel('time')
        axs[1].set_ylabel('[metabolite]');

def plot_fit_gMLV(yobs, yobs_h, sobs, sobs_h, timepoints):
    # plot the fit
    #fig, axs = plt.subplots(1, 2, layout='constrained')
    fig, axs = plt.subplots(1, 2)  
    for species_idx in range(yobs.shape[1]):
        axs[0].plot(timepoints, yobs[:, species_idx], color=cols[species_idx])
        axs[0].plot(timepoints, yobs_h[:, species_idx], '--', color=cols[species_idx])
    axs[0].set_xlabel('time')
    axs[0].set_ylabel('[species]');

    for metabolite_idx in range(sobs.shape[1]):
        axs[1].plot(timepoints, sobs[:, metabolite_idx], color=cols[metabolite_idx])
        axs[1].plot(timepoints, sobs_h[:, metabolite_idx], '--', color=cols[metabolite_idx])
    axs[1].set_xlabel('time')
    axs[1].set_ylabel('[metabolite]');

def compare_params(mu=None, M=None, alpha=None, e=None):
    # each argument is a tuple of true and predicted values
    if mu is not None:
        print("mu_hat/mu:")
        print(np.array(mu[1]))
        print(np.array(mu[0]))

        fig, ax = plt.subplots()
        ax.stem(np.arange(0,len(mu[0]), dtype="int32"), np.array(mu[1]), markerfmt="D")
        ax.stem(np.arange(0,len(mu[0]), dtype="int32"), np.array(mu[0]), markerfmt="X")
        ax.set_xlabel('i')
        ax.set_ylabel('mu[i]');

    if M is not None:
        print("\nM_hat/M:")
        print(np.round(np.array(M[1]), decimals=2))
        print("\n",np.array(M[0]))

        #fig, ax = plt.subplots()
        #ax.stem(np.arange(0, M[0].shape[0] ** 2), np.array(M[1]).flatten(), markerfmt="D")
        #ax.stem(np.arange(0, M[0].shape[0] ** 2), np.array(M[0]).flatten(), markerfmt="X")
        #ax.set_ylabel('M[i,j]');

        fig, ax = plt.subplots()
        Ns = M[0].shape[0]
        ax.stem(np.arange(0, Ns), np.array(M[1]).diagonal(), markerfmt="D")
        ax.stem(np.arange(0, Ns), np.array(M[0]).diagonal(), markerfmt="X")
        ax.set_ylabel('M[i,i]');
        
        fig, ax = plt.subplots()
        Ns = M[0].shape[0]
        
        count = 0
        Mij = np.zeros([Ns*Ns - Ns])
        Mij_h = np.zeros([Ns*Ns - Ns])
        for i in range(Ns):
            for j in range(Ns):
                if i != j:
                    Mij[count] = np.array(M[0])[i,j]
                    Mij_h[count] = np.array(M[1])[i,j]
                    count = count + 1
        
        ax.stem(np.arange(0, Ns*Ns - Ns), Mij.flatten(), markerfmt="D")
        ax.stem(np.arange(0, Ns*Ns - Ns), Mij_h.flatten(), markerfmt="X")
        ax.set_ylabel('M[i,i]');
        
    if alpha is not None:
        print("\na_hat/a:")
        print(np.round(np.array(alpha[1]), decimals=2))
        print("\n",np.array(alpha[0]))

        fig, ax = plt.subplots()
        ax.stem(np.arange(0, alpha[0].shape[0] * alpha[0].shape[1]), np.array(alpha[1]).flatten(), markerfmt="D")
        ax.stem(np.arange(0, alpha[0].shape[0] * alpha[0].shape[1]), np.array(alpha[0]).flatten(), markerfmt="X")
        ax.set_ylabel('a[i,j]');

    if e is not None:
        print("\ne_hat/e:")
        print(np.round(np.array(e[1]), decimals=2))
        print("\n",np.array(e[0]))

        fig, ax = plt.subplots()
        ax.stem(np.arange(0, e[0].shape[0]), np.array(e[1]).flatten(), markerfmt="D")
        ax.stem(np.arange(0, e[0].shape[0]), np.array(e[0]).flatten(), markerfmt="X")
        ax.set_ylabel('e[i]');


In [ ]:
# some MCMC analysis functions

def make_trace_plot(var,istart,iend):
    plt.figure()
    post = df[var][istart:iend]
    plt.plot(range(0,(iend-istart)),post)
    #print(var, np.median(post))
    return
    
def make_hist_plot(var,istart,iend):
    plt.figure()
    post = df[var][istart:iend]
    plt.hist(post)
    print(var, np.median(post))
    return
    
def get_Rhat(N,p1,p2):
    M = 2
    mean1 = np.mean(p1,axis=0)  
    mean2 = np.mean(p2,axis=0)  
    var1 = np.var(p1,axis=0)  
    var2 = np.var(p2,axis=0)
    
    meanM = (1/M)*(mean1 + mean2)
    
    B = (N/(M-1)) * (mean1-meanM)*(mean1-meanM) + (mean2-meanM)*(mean2-meanM)
    W = (1/M)*(var1 + var2)
    
    Vhat = ((N-1)/N)*W + ((M+1)/(M*N))*B
    
    Rhat = Vhat/W
    
    return Rhat

def get_horseshoe_tau(p0,D,sigma,n):
    return p0*sigma/( np.sqrt(n)*(D-p0) )

# Single time course: three species

In [ ]:
set_all_seeds(1234)

## SETUP MODEL
# establish size of model
num_species = 3
num_metabolites = 0

# construct interaction matrix
M = np.zeros((num_species, num_species))
np.fill_diagonal(M, [-0.05, -0.1, -0.15])
M[0, 2] = -0.1
M[2, 0] = 0.1

# construct growth rates matrix
mu = np.random.lognormal(0.01, 0.5, num_species)

# instantiate simulator
simulator = gMLV_sim(num_species=num_species,
                     num_metabolites=num_metabolites,
                     M=M,
                     mu=mu)
simulator.print()

## PRODUCE SIMULATED RESULTS
# initial conditions
init_species = 10 * np.ones(num_species)
init_metabolites = 10 * np.ones(num_metabolites)

times = np.arange(0, 5, 0.1)
yobs, sobs, sy0, mu, M, _ = simulator.simulate(times=times, sy0=np.hstack((init_species, init_metabolites)))

# add some gaussian noise
yobs_x = yobs + np.random.normal(loc=0, scale=0.1, size=yobs.shape)
plot_gMLV(yobs_x, sobs, times)

# add some gaussian noise on log scale
#yobs_lnx = np.log(yobs) + np.random.normal(loc=0, scale=0.01, size=yobs.shape)
#plot_gMLV(yobs_lnx, sobs, times)


## Full Bayesian inference using ODEs

In [ ]:
# Prior visualisation
# mu
x_mu = np.random.lognormal(0.01,0.5,size=10000)
plt.figure()
plt.hist(x_mu);
plt.title('mu')   

# Md
x_Md = np.random.normal(0.1,0.05,size=10000)
plt.figure()
plt.hist(x_Md);
plt.title('M[i,i]')    
    
# Shrinkage, M
tau0 = 0.001;
x_M = np.zeros([1000])
for i in range(1000):
    tau = np.random.standard_cauchy(size=1)
    lam = np.random.standard_cauchy(size=1)
    x_M[i] = np.random.normal(0,np.abs(lam)*np.abs(tau),size=1)
#print(x_M)
plt.figure()
plt.hist(x_M, range=(-5,5));
plt.title('M[i,j]')   

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import stan

gLV_code = """
functions {
   vector lotka_volterra(real t, vector y, vector mu, vector Md, vector M) {
     vector[2] dydt;
     dydt[1] = mu[1]*y[1] - Md[1]*y[1]*y[1] + M[1]*y[1]*y[2];
     dydt[2] = mu[2]*y[2] - Md[2]*y[2]*y[2] + M[2]*y[2]*y[1];
     return dydt;
  }

  vector lotka_volterra_dln(real t, vector y, vector mu, vector Md, vector M) {
     vector[2] dydt;
     dydt[1] = mu[1] - Md[1]*y[1] + M[1]*y[2];
     dydt[2] = mu[2] - Md[2]*y[2] + M[2]*y[1];
     return dydt;
  }

  vector lotka_volterra_N(real t, vector y, int N, vector mu, vector Md, vector M) {
     vector[N] dydt;
     
     int countM = 1;
     
     for(i in 1:N){
        dydt[i] = mu[i]*y[i] - Md[i]*y[i]*y[i];
        
        for(j in 1:N){
            if ( i != j ){
                dydt[i] += M[countM]*y[i]*y[j];
                countM += 1; 
                //print("loop iteration: ", i, j, countM);
             }
         }
     }
     
     return dydt;
  }


}

data {
  int<lower=1> N;
  int<lower=1> T;
  real t0;
  array[T] real ts;
  array[T,N] real y;
  vector[N] y0;
  real sigma;
  //real tau;
  
  //vector[2] M;
  //vector[2] Md;
}

parameters {
  vector<lower=0>[N]  mu;
  vector<lower=0>[N]  Md;
  vector[N*N - N]     M;
   
  vector<lower=0>[N*N - N]  lambda;
  real<lower=0>  tau;
  
  //vector[N]  y0;
  //real<lower=0>     sigma;
}

model {
  //target += double_exponential_lpdf(mu | 0, 1.0);
  //target += double_exponential_lpdf(Md | 0, 0.1);
  
  //target += normal_lpdf(mu | 1.0, 0.2);
  target += lognormal_lpdf(mu | 0.01, 0.5);
  
  target += normal_lpdf(Md | 0.1, 0.02);
  
  // Laplace
  //target += double_exponential_lpdf(M | 0, 0.1);

  // exponential / normal
  //target += exponential_lpdf(lambda | 10);
  //target += normal_lpdf(M | 0, lambda);

  // hierarchical exponential
  //for(i in 1:2){
  //    target += exponential_lpdf(lambda[i] | 10); // parameterised as 1/scale
  //    target += normal_lpdf(M[i] | 0, lambda[i]);
  //}
  
  // Horsehoe prior
  real tau0 = 0.001;
  target += cauchy_lpdf(tau | 0, tau0);

  for(i in 1:(N*(N-1))){
        target += normal_lpdf(M[i] | 0, lambda[i]*tau);
        target += cauchy_lpdf(lambda[i] | 0, 1);
  }
  
  vector[N] y_hat[T] = ode_bdf_tol(lotka_volterra_N, y0, t0, ts, 1e-6, 1e-6, 100000, N, mu, Md, M );

  for (t in 1:T) {
      for (s in 1:N){
        target += normal_lpdf(y[t,s] | y_hat[t,s],sigma);
      }
    }
}

"""

#obs_data_log = {"T": len(times)-1,
#                "t0": 0.0,
#                "ts": times[1:],   
#                "y": yobs_lnx[1:,:],
#                "y0": np.log(init_species),
#                "sigma": 0.01,
#                #"Md": np.array([-M[0,0],-M[1,1] ]),
#                #"M": np.array( [M[0,1],M[1,0]] )
#               } 

obs_data = {"N": 3,
            "T": len(times)-1,
            "t0": 0.0,
            "ts": times[1:],   
            "y": yobs_x[1:,:],
            "y0": init_species,
            "sigma": 0.1,
            #"tau:": 1.0
            #"Md": np.array([-M[0,0],-M[1,1] ]),
            #"M": np.array( [M[0,1],M[1,0]] )
           } 

#posterior = stan.build(gLV_code, data=obs_data_log, random_seed=1)
posterior = stan.build(gLV_code, data=obs_data, random_seed=1)

In [ ]:
sample_kwargs = {"num_samples": 500, "num_chains": 2, "num_warmup": 5000 }
#fit = posterior.sample(num_chains=2, num_samples=500, num_warmup=10000, adapt={'delta':0.99})
fit = posterior.sample(**sample_kwargs)

#print(fit)

df = fit.to_frame()
print(df.describe().T)
#print(df.head())

post1 = df[df.columns[7:]][0:500].to_numpy()
post2 = df[df.columns[7:]][500:1000].to_numpy()
#post1 = np.random.normal(size=500)
#post2 = np.random.normal(size=500)
Rhat = get_Rhat(500, post1, post2)
print("Rhat:", Rhat)
istart = 0
iend = 1000

In [ ]:
print("mu:",mu)
for i in range(num_species):
    make_trace_plot("mu."+str(i+1),istart,iend)
    make_hist_plot("mu."+str(i+1),istart,iend)


print("Md:",M.diagonal())
for i in range(num_species):
    make_trace_plot("Md."+str(i+1),istart,iend)
    make_hist_plot("Md."+str(i+1),istart,iend)


for i in range( num_species*(num_species - 1)):
    make_trace_plot("M."+str(i+1),istart,iend)
    make_hist_plot("M."+str(i+1),istart,iend)


In [ ]:
# plot the fit using median values of parameters

post1 = df[df.columns[7:]][0:iend].to_numpy()
est = np.median(post1,axis=0)

# fill mu_h and M-H
mu_h = est[0:num_species]
M_h = np.zeros([num_species,num_species])
np.fill_diagonal(M_h, -est[num_species:2*num_species])

count = 0
print("est:", est)
for i in range(num_species):
    for j in range(num_species):
        if i != j:
            M_h[i,j] = est[2*num_species + count]
            count = count + 1

#print(mu_h)
#print(M_h)

# make the plot and comparison
predictor = gMLV_sim(num_species=num_species,
                     num_metabolites=num_metabolites,
                     M=M_h,
                     mu=mu_h)
yobs_h, sobs_h, _, _, _, _ = predictor.simulate(times=times, sy0=np.hstack((init_species, init_metabolites)))

## PLOT RESULTS
# plot comparison of simulated and predicted timeseries
plot_fit_gMLV(yobs, yobs_h, sobs, sobs_h, times)

# this does the stem plots with orange crosses the actual parameters
compare_params(mu=(mu,mu_h), M=(M, M_h))